In [ ]:
using Comrade # Import Comrade
using Pyehtim # Load eht-imaging (used for data io)

In [ ]:
using StableRNGs
rng = StableRNG(42)

In [ ]:
obs = ehtim.obsdata.load_uvfits(joinpath(dirname(pathof(Comrade)), "..", "examples", "SR1_M87_2017_096_lo_hops_netcal_StokesI.uvfits"))
obs = Pyehtim.scan_average(obs.flag_uvdist(uv_min=0.1e9))
typeof(obs)

In [ ]:
dlcamp, dcphase = extract_table(obs, LogClosureAmplitudes(;snrcut=3), ClosurePhases(;snrcut=3))

In [ ]:
function model(θ)
    (;radius, width, α, β, f, σG, τG, ξG, xG, yG) = θ
    ring = f*smoothed(stretched(MRing((α,), (β,)), radius, radius), width)
    g = (1-f)*shifted(rotated(stretched(Gaussian(), σG, σG*(1+τG)), ξG), xG, yG)
    return ring + g
end

In [ ]:
lklhd = RadioLikelihood(model, dlcamp, dcphase)

In [ ]:
using Distributions, DistributionsAD
prior = (
        radius = Uniform(μas2rad(10.0), μas2rad(30.0)),
        width = Uniform(μas2rad(1.0), μas2rad(10.0)),
        α = Uniform(-0.5, 0.5),
        β = Uniform(-0.5, 0.5),
        f = Uniform(0.0, 1.0),
        σG = Uniform(μas2rad(1.0), μas2rad(40.0)),
        τG = Uniform(0.0, 0.75),
        ξG = Uniform(0.0, 1π),
        xG = Uniform(-μas2rad(80.0), μas2rad(80.0)),
        yG = Uniform(-μas2rad(80.0), μas2rad(80.0))
)

In [ ]:
post = Posterior(lklhd, prior);

In [ ]:
logdensityof(post, (radius = μas2rad(20.0),
                  width = μas2rad(10.0),
                  α = 0.3,
                  β = 0.3,
                  f = 0.6,
                  σG = μas2rad(20.0),
                  τG = 0.1,
                  ξG = 0.5,
                  xG = 0.0,
                  yG = 0.0))

In [ ]:
# Construct a transform that moves the parameters from the unit
# hypercube to what the parameters we specified.
cpost = ascube(post) 

In [ ]:
# Construct a transform that moves the parameters from unconstrained
# R^N to the parameters space.
fpost = asflat(post)

In [ ]:
logdensityof(cpost, rand(rng, dimension(cpost))) # notice we use rand here since the argument must be in [0,1]
logdensityof(fpost, randn(rng, dimension(fpost))) # notice we use randn since the argument can be all of R^N

In [ ]:
#, we will use `cpost` since it restricts the domain to live within the compact unit hypercube
#, which is easier to explore for non-gradient-based optimizers like `BBO`.
using ComradeOptimization
using OptimizationBBO

ndim = dimension(cpost)
f = OptimizationFunction(cpost)
prob = Optimization.OptimizationProblem(f, randn(rng, ndim), nothing, lb=fill(0.0, ndim), ub=fill(1.0, ndim))

In [ ]:
sol = solve(prob, BBO_adaptive_de_rand_1_bin_radiuslimited(); maxiters=5_00)

In [ ]:
xopt = transform(cpost, sol)

In [ ]:
using Plots
Plots.plot(model(xopt), title="MAP image", xlims=(-60.0,50.0), ylims=(-60.0,50.0))

In [ ]:
using ComradeAHMC, ForwardDiff
chain, stats = sample(rng, post, AHMC(metric=DiagEuclideanMetric(ndim), autodiff=Val(:ForwardDiff)), 2000; nadapts=1000, init_params=xopt)

In [ ]:
Plots.plot(model(chain[end]), title="Random image")

In [ ]:
meanimg = mean(intensitymap.(model.(sample(chain[1000:end], 100)), μas2rad(120.0), μas2rad(120.0), 128, 128))
Plots.plot(sqrt.(max.(meanimg, 0.0)), title="Mean Image") #plot on a sqrt color scale to see the Gaussian

In [ ]:
Plots.plot(model(xopt), dlcamp, label="MAP")